In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

In [2]:
with open("60607_apartments.csv", "w") as file:
    file.write(
        "Address,Bedrooms,Bathrooms,Rent,Total Area,Price per Sq.Ft.,Transit Score,Type of House\n")

In [27]:
new_data = []

In [25]:
with open("60607_apartments_link.csv", encoding="utf-8") as file:
    links = file.readlines()

links = set(links)

In [28]:
len(links)

699

In [29]:

with open("60607_apartments_link.csv", encoding="utf-8") as file:
    links = file.readlines()
    for link in links:
        try:
            url = link.strip()
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

            response = requests.get(url, headers=headers)
            html_content = response.content

            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')

            if not soup:
                continue
            property_name = soup.find('h1', class_="propertyName")
            # Find the <div> element with class "propertyAddressRow"
            address_row = soup.find('div', class_='propertyAddressRow')

            # Extract the text from all child elements of the <div> element
            address_text = property_name.get_text(
                strip=True) + " " + address_row.get_text(strip=True)

            # Find the <ul> element with class "priceBedRangeInfo"
            price_info = soup.find('ul', class_='priceBedRangeInfo')

            # Extract minimum and maximum rent
            rent_detail = price_info.find(
                'p', class_='rentInfoDetail').get_text().split(' - ')
            rent = int(rent_detail[0].strip('$').replace(',', ''))

            # Extract number of bedrooms
            bedrooms_detail = price_info.find_all(
                'p', class_='rentInfoDetail')[1].get_text()
            num_bedrooms = bedrooms_detail.split(' ')[-2]

            # Extract number of bathrooms
            bathrooms_detail = price_info.find_all(
                'p', class_='rentInfoDetail')[2].get_text()
            num_bathrooms = bathrooms_detail.split(' ')[-2]

            # Extract minimum and maximum square feet
            sq_ft_detail = price_info.find_all('p', class_='rentInfoDetail')[
                3].get_text().split(' - ')
            sq_ft = sq_ft_detail[0].replace(',', '').strip(' sq ft').strip()

            if sq_ft:
                price_per_sq_ft = rent/int(sq_ft)
            else:
                price_per_sq_ft = 0

            residence_type = ""

            transit_score = soup.find('div', class_='transitScore').find(
                'div', class_='score').text
            
            new_data.append([address_text,num_bedrooms,num_bathrooms,rent,sq_ft,price_per_sq_ft,transit_score,residence_type])
        except:
            pass


In [30]:
len(new_data)

696

In [17]:
df = pd.DataFrame(appartment_data)
df

,0,1,2,3,4,5,6,7
0,"123 S Green St Unit 910B Chicago,IL60607–West ...",1,1,2300,800,2.875000,90,
1,"707 W Wayman St Chicago,IL60661–West Loop",2,2,3400,,0.000000,87,
2,"111 S Peoria St Unit 403 Chicago,IL60607–Fulto...",3,3.5,7600,,0.000000,88,
3,"719 S Loomis St Unit GRD Chicago,IL60607–Littl...",3,1,2000,1300,1.538462,70,
4,"948 W Madison St Unit 5NW Chicago,IL60607–West...",2,2,3200,1025,3.121951,84,
...,...,...,...,...,...,...,...,...
826,"111 W Polk St Unit FL10-ID688 Chicago,IL60605–...",1,1,3130,752,4.162234,100,
827,"1380 W Randolph St Unit 1 Chicago,IL60607–Fult...",1,1,2359,550,4.289091,72,
828,"700 S Clark St 700 S Clark St,Chicago,IL60605–...",2,2,2290,735,3.115646,100,
829,"933 W Van Buren St Unit 816 Chicago,IL60607–We...",2,2,3400,1211,2.807597,82,


In [18]:
df.tail(10)

,0,1,2,3,4,5,6,7
821,"901 W Madison St Unit 404 Chicago,IL60607–Fult...",1,1.5,2500,900,2.777778,88,
822,"1250 W Van Buren St Unit 205 Chicago,IL60607–W...",2,1,2300,100,23.000000,74,
823,"333 S Desplaines St Unit 505 Chicago,IL60661–W...",1,1,2300,1000,2.300000,97,
824,"1116 W Polk St Unit 1RB Chicago,IL60607–Univer...",1,1,1400,900,1.555556,73,
825,"1225 W Cottage Plz Chicago,IL60607–West Loop",3,3,3950,1850,2.135135,75,
826,"111 W Polk St Unit FL10-ID688 Chicago,IL60605–...",1,1,3130,752,4.162234,100,
827,"1380 W Randolph St Unit 1 Chicago,IL60607–Fult...",1,1,2359,550,4.289091,72,
828,"700 S Clark St 700 S Clark St,Chicago,IL60605–...",2,2,2290,735,3.115646,100,
829,"933 W Van Buren St Unit 816 Chicago,IL60607–We...",2,2,3400,1211,2.807597,82,
830,"6 S Laflin St Unit 522 Chicago,IL60607–Fulton ...",1,1,2100,,0.000000,76,


In [19]:
df.drop_duplicates()

,0,1,2,3,4,5,6,7
0,"123 S Green St Unit 910B Chicago,IL60607–West ...",1,1,2300,800,2.875000,90,
1,"707 W Wayman St Chicago,IL60661–West Loop",2,2,3400,,0.000000,87,
2,"111 S Peoria St Unit 403 Chicago,IL60607–Fulto...",3,3.5,7600,,0.000000,88,
3,"719 S Loomis St Unit GRD Chicago,IL60607–Littl...",3,1,2000,1300,1.538462,70,
4,"948 W Madison St Unit 5NW Chicago,IL60607–West...",2,2,3200,1025,3.121951,84,
...,...,...,...,...,...,...,...,...
826,"111 W Polk St Unit FL10-ID688 Chicago,IL60605–...",1,1,3130,752,4.162234,100,
827,"1380 W Randolph St Unit 1 Chicago,IL60607–Fult...",1,1,2359,550,4.289091,72,
828,"700 S Clark St 700 S Clark St,Chicago,IL60605–...",2,2,2290,735,3.115646,100,
829,"933 W Van Buren St Unit 816 Chicago,IL60607–We...",2,2,3400,1211,2.807597,82,


In [32]:
df = pd.DataFrame(new_data)
df.to_csv("data.csv", index=False)

In [33]:
df = pd.read_csv("data.csv")
df

,Address,Bedrooms,Bathrooms,Rent,Total Area,Price per Sq.Ft.,Transit Score,Type of House
0,"123 S Green St Unit 910B Chicago,IL60607–West ...",1,1.0,2300,800.0,2.875000,90,NaN
1,"707 W Wayman St Chicago,IL60661–West Loop",2,2.0,3400,NaN,0.000000,86,NaN
2,"111 S Peoria St Unit 403 Chicago,IL60607–Fulto...",3,3.5,7600,NaN,0.000000,88,NaN
3,"719 S Loomis St Unit GRD Chicago,IL60607–Littl...",3,1.0,2000,1300.0,1.538462,70,NaN
4,"948 W Madison St Unit 5NW Chicago,IL60607–West...",2,2.0,3200,1025.0,3.121951,83,NaN
...,...,...,...,...,...,...,...,...
691,"125 S Green St 125 S Green St,Chicago,IL60607–...",2,2.0,2500,900.0,2.777778,90,NaN
692,"111 W Polk St Unit FL2-ID690 Chicago,IL60605–S...",1,1.0,3110,736.0,4.225543,100,NaN
693,"170 W Polk St 170 W Polk St,Chicago,IL60605–So...",2,2.0,1874,522.0,3.590038,100,NaN
694,"125 S Green St Unit 1109A Chicago,IL60607–West...",1,1.0,2500,800.0,3.125000,90,NaN
